# Deep Learning with Python
# 5.3.3 - Visualizing Heatmaps of Class Activation

- Useful for debugging CV applications because it identifies which parts of an input image led the convnet to its final classification decision.
- Also useful for locating specific objects in the image.
- Belongs to the general category of techniques called *class activation maps*. 
- 2D grid of score sassociated with a specific output class that is computed for every location in an any input image.
- Indicates how important each locatio is with repsect ot the class under consideration. 

## GRAD-CAM
- Takes the output feature map of a convolution layer, given an input image, and weighs eveyr channel in that feature map by the gradient of the class with respect to the channel.

$$\frac{\delta[image activation]}{\delta[channel]} \times \frac{\delta[channel]}{\delta[class]} = \frac{\delta[image activation]}{\delta[class]}$$


## Predicting with `ImageNet` Classifier

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16

In [3]:
model = VGG16(weights='imagenet', include_top=False)

## Preprocessing Image

In [14]:
# Module for creating image tensors
from tensorflow.keras.preprocessing import image

In [7]:
# VGG 16's own rules for preprocessing images and predicting the output class
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [8]:
import numpy as np

In [9]:
# Image of elephants
img_path = '/Users/saads/OneDrive/Desktop/DL-Python/chapter-5/ch-5-repeat/creative_commons_elephant.jpg'

In [10]:
# Load from file as a PIL object
img = image.load_img(img_path, target_size=(224, 244))

In [11]:
# Conver to numpy-compatible array
x = image.img_to_array(img)

In [15]:
# Expand shape of the array and convert to tensor by adding batch axis
x = np.expand_dims(x, axis=0)

In [16]:
# Use VGG 16's own rules to process the input
x = preprocess_input(x)

## Predictions

In [17]:
preds = model.predict(x)

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (1, 1, 224, 244, 3)

## Grad-CAM Process

In [ ]:
# Output will be a 385 dimensional vector encoding probability
# of image belonging to one of 385 classes
african_elephant_output = model.output[: 386]

In [18]:
# Output feature map fo the last convolutional layer in VGG16
last_conv_layer = model.get_layer('block5_conv3')

In [19]:
import tensorflow.keras.backend as K

In [ ]:
# Gradient of the African Elephatn class w.r.t output feature map of conv layer
grads = K.gradients(african_elephant_output,
3                   last_conv_layer.output)[0]

In [ ]:
# Vector of shape (512, ) where each entry is mean intensity of the gradient
# over a specific feature-map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

In [ ]:
# Define a function that gets the gradient for all filters
# As well as the conv layer output using the model input
iterate = K.function([model.input], 
                    [pooled_grads, last_conv_layer.output[0]])

In [ ]:
# Access the quantities we just defined 
pooled_grads_value, conv_layer_output_value = iterate(x)

In [ ]:
# Multiply each channel in the feature map array by
# "how important this channel is" w.r.t the `elephant` class
for i in range(512):
    conv_layer_output_vaalue[:, :, 1] *= pooled_grads_value[i]

In [ ]:
# The channel-wise mean of the resulting feature map is the heatmap of the class activation
heatmap = np.mean(conv_layer_output_value, axis=1)

In [ ]:
# Normalize the heatmpa between 0 and 1

# Compute element-wise maxima of matrix and 0, ensuring any
# values below 0 will be set to 0 in the new heatmap
heatmap = np.maximum(heatmap, 0)        

# Then normalize the heatmap by dividing all values by the max value in the map
heatmap /= np.max(heatmap) 

# Render
plt.matshow(heatmap)

## `OpenCV` for Superimposition

Using `OpenCV` to superimpose the original image on the heatmap.

In [ ]:
import cv2

In [ ]:
# Use Open CV to load the original image
img = cv2.imread(img_path)

In [ ]:
# Resize heatmap to be the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

In [ ]:
# Convert heatmap to an RGB array
heatamp = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

In [ ]:
superimposed_img = heatmap * 0.4 + img

In [ ]:
cv2.imrwrite('/Users/saads/OneDrive/Desktop/DL-Python/chapter-5/ch-5-repeat/elephant_cam.jpg', 
            superimposed_img)